# Setup

## Pip install

In [1]:
# Don't forget to restart runtime after installing

%pip install "labelbox[data]" --quiet  # installs all required libraries plus extras required in manipulating annotations (shapely, geojson, numpy, PILLOW, opencv-python, etc.)

# %pip freeze
# %pip freeze | grep matplotlib  # get version

     |████████████████████████████████| 173 kB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 25.7 MB/s 


## Base imports


In [2]:
import os
import sys
print(sys.version)
import json
import numpy as np
import pandas as pd
import scipy
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import skimage
import skimage.io
#from PIL import Image
import PIL
import PIL.Image
import requests

import labelbox
#from labelbox.data.annotation_types import Geometry

import IPython.display
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [3]:
notebook_filename = requests.get("http://172.28.0.2:9000/api/sessions").json()[0]["name"]

# Avoids scroll-in-the-scroll in the entire Notebook
def resize_colab_cell():
  display(IPython.display.Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 10000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)


#@markdown ### func `def get_path_to_save(...):`
def get_path_to_save(plot_props:dict=None, file_prefix="", save_filename:str=None, save_in_subfolder:str=None, extension="jpg", dot=".", create_folder_if_necessary=True):
    """
    Code created myself (Rahul Yerrabelli)
    """
    replace_characters = {
        "$": "",
        "\\frac":"",
        "\\mathrm":"",
        "\\left(":"(",
        "\\right)":")",
        "\\left[":"[",
        "\\right]":"]",
        "\\": "",
        "/":"-",
        "{": "(",
        "}": ")",
        "<":"",
        ">":"",
        "?":"",
        "_":"",
        "^":"",
        "*":"",
        "!":"",
        ":":"-",
        "|":"-",
        ".":"_",
    }

    # define save_filename based on plot_props
    if save_filename is None:
        save_filename = "unnamed"

    #save_path = f"../outputs/{notebook_filename.split('.',1)[0]}"
    save_path = [
                 "outputs",
                f"{notebook_filename.split('.',1)[0]}",
                ]
    if save_in_subfolder is not None:
        if isinstance(save_in_subfolder, (list, tuple, set, np.ndarray) ):
            save_path.append(**save_in_subfolder)
        else:  # should be a string then
            save_path.append(save_in_subfolder)
    save_path = os.path.join(*save_path)

    if not os.path.exists(save_path) and create_folder_if_necessary:
        os.makedirs(save_path)
    return os.path.join(save_path, file_prefix+save_filename+dot+extension)
    #plt.savefig(os.path.join(save_path, save_filename+dot+extension))



In [4]:
#@title ## Mount google drive and import my code

mountpoint_folder_name = "drive"  # can be anything, doesn't have to be "drive"
project_path_within_drive = "PythonProjects/SpeculumAnalysis" #@param {type:"string"}
#project_path_within_drive = "UIUC ECs/Rahul_Ashkhan_Projects/SpeculumProjects_Shared/Analysis" #@param {type:"string"}
project_path_full = os.path.join("/content/",mountpoint_folder_name,
                        "MyDrive",project_path_within_drive)

%cd {project_path_full}

<IPython.core.display.Javascript object>

[Errno 2] No such file or directory: '/content/drive/MyDrive/PythonProjects/SpeculumAnalysis'
/content


In [5]:
try:
    import google.colab.drive
    import os, sys
    # Need to move out of google drive directory if going to remount
    %cd
    # drive.mount documentation can be accessed via: drive.mount?
    #Signature: drive.mount(mountpoint, force_remount=False, timeout_ms=120000, use_metadata_server=False)
    google.colab.drive.mount(os.path.join("/content/",mountpoint_folder_name), force_remount=True)  # mounts to a folder called mountpoint_folder_name

    if project_path_full not in sys.path:
        pass
        #sys.path.insert(0,project_path_full)
    %cd {project_path_full}
    
except ModuleNotFoundError:  # in case not run in Google colab
    import traceback
    traceback.print_exc()

<IPython.core.display.Javascript object>

/root
Mounted at /content/drive
/content/drive/MyDrive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/SpeculumAnalysis


# Data

## Read in the collected/labeled data

### Labelbox

#### Option 1: Read from labelbox

##### Set up labelbox connection
Works with LabelBox api (https://labelbox.com/), which is the tool I used to label all the distances on the images.

In [6]:
# Add your labelbox api key and project
# Labelbox API stored in separate file since it is specific for a labelbox 
#account and shouldn't be committed to git. Contact the 
# team (i.e. Rahul Yerrabelli) in order to access to the data on your own account.
with open("auth/LABELBOX_API_KEY.json", "r") as infile:
  json_data = json.load(infile)
API_KEY = json_data["API_KEY"]
del json_data   # delete sensitive info

PROJECT_ID = "cl2cept1u4ees0zbx6uan5kwa"
DATASET_ID_Glove = "cl2cerkwd5gtd0zcahfz98401"; DATASET_NAME_Glove = "SpeculumWithGlove"
DATASET_ID_Condom = "cl2hu1u8z019a0z823yl5f8gr"; DATASET_NAME_Condom = "SpeculumWithCondom"

client = labelbox.Client(api_key=API_KEY)
del API_KEY   # delete sensitive info
project = client.get_project(PROJECT_ID)
dataset_glove = client.get_dataset(DATASET_ID_Glove)
dataset_condom = client.get_dataset(DATASET_ID_Condom)



# Alternative way to get dataset
# dataset = next(client.get_datasets(where=(labelbox.Dataset.name == DATASET_NAME)))

# Below code is from labelbox tutorial
# Create a mapping for the colors
hex_to_rgb = lambda hex_color: tuple(
    int(hex_color[i + 1:i + 3], 16) for i in (0, 2, 4))
colors = {
    tool.name: hex_to_rgb(tool.color)
    for tool in labelbox.OntologyBuilder.from_project(project).tools
}

<IPython.core.display.Javascript object>

In [7]:
image_labels = project.label_generator()
image_labels = image_labels.as_list()
labels_df = pd.DataFrame([[
                           label.data.external_id, 
                           label.annotations[0].value.end.x - label.annotations[0].value.start.x, 
                           label.annotations[0].value.end.y - label.annotations[0].value.start.y, 
                           label.annotations[0].value.start.x, 
                           label.annotations[0].value.start.y, 
                           label.data.url, 
                           label.uid
                           ] 
                          for label in image_labels],
                         columns=["Filename","x","y", "xstart","ystart","url", "Label ID"])

labels2 = project.export_labels(download = True, start="2022-04-01", end="2022-06-01")
labels3 = [value.copy() for value in labels2 ]

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/labelbox/data/annotation_types/classification/classification.py:85: UserWarning: Dropdown classification is deprecated and will be removed in a future release
  warnings.warn("Dropdown classification is deprecated and will be "


In [ ]:
labels2

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

#### Save mask images 

In [ ]:
% ls dist/dataset/photos

20220423_142023.jpg        20220423_153412.jpg        20220423_164749.jpg
20220423_142023_label.png  20220423_153412_label.png  20220423_164749_label.png
20220423_142031.jpg        20220423_153417.jpg        20220423_164754.jpg
20220423_142031_label.png  20220423_153417_label.png  20220423_164754_label.png
20220423_142049.jpg        20220423_153422.jpg        20220423_164759.jpg
20220423_142049_label.png  20220423_153422_label.png  20220423_164759_label.png
20220423_142054.jpg        20220423_153426.jpg        20220423_164803.jpg
20220423_142054_label.png  20220423_153426_label.png  20220423_164803_label.png
20220423_142100.jpg        20220423_153430.jpg        20220423_164808.jpg
20220423_142100_label.png  20220423_153430_label.png  20220423_164808_label.png
20220423_142108.jpg        20220423_153434.jpg        20220423_164812.jpg
20220423_142108_label.png  20220423_153434_label.png  20220423_164812_label.png
20220423_143310.jpg        20220423_153847.jpg        20220423_165244.jpg
20

In [ ]:
# Export labels created in the selected date range as a json-type (list of dicts/elements/lists):
labels2 = project.export_labels(download = True, start="2022-04-01", end="2022-06-01")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
labels3 = [value.copy() for value in labels2 ] # copy at least one level deep

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for ind in range(len(labels3)):
    # Simplify "Label" and "Reviews" by removing unnecessary variables and making the necessary ones at the top level
    # Thus, labels3 will be only 2 layers deep.
    if "Label" in labels3[ind]:
        coords = labels3[ind]["Label"]["objects"][0]["bbox"]
        for key, val in coords.items():
            labels3[ind]["Label"+"-"+key] = val
        # URL to download mask. Still has token in it
        labels3[ind]["Label_url"] = labels3[ind]["Label"]["objects"][0]["instanceURI"] 
        del labels3[ind]["Label"]

    # Remove special info ie emails, tokens (except the Label_url for now)
    labels3[ind].pop("Labeled Data", None)  # url with token in it
    labels3[ind].pop("View Label", None)  # url
    labels3[ind].pop("Created By", None)  # has email address
    labels3[ind].pop("Reviews", None)  # empty list

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Download images from URLs
import urllib.request

for ind in range(len(labels3)):
    filename = labels3[ind]["External ID"].split(".jpg")[0] + "_label"
    filepath = get_path_to_save(save_filename=filename, extension="png")
    urllib.request.urlretrieve(labels3[ind]["Label_url"], filepath)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Remove Label_url as that URL has the Labelbox token in it
labelbox_df = pd.DataFrame.from_dict(labels3).set_index("External ID").drop(columns=["Label_url"])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
labelbox_df.to_csv( get_path_to_save(save_filename="labelbox_details", extension="csv") )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### Get dataframe now that labelbox is set up

In [ ]:
labels_df

In [ ]:
image_labels = project.label_generator()
image_labels = image_labels.as_list()
labels_df = pd.DataFrame([[
                           label.data.external_id, 
                           label.annotations[0].value.end.x - label.annotations[0].value.start.x, 
                           label.annotations[0].value.end.y - label.annotations[0].value.start.y, 
                           label.annotations[0].value.start.x, 
                           label.annotations[0].value.start.y, 
                           label.data.url, 
                           label.uid
                           ] 
                          for label in image_labels],
                         columns=["Filename","x","y", "xstart","ystart","url", "Label ID"])
labels_df.to_csv("data/02_intermediate/labels_df"+".csv")
labels_df.to_pickle("data/02_intermediate/labels_df"+".pkl")

label_from_id_dict = {label.data.external_id: label for label in image_labels}
#with open("data/02_intermediate/label_from_id_dict"+".json", "w") as outfile:
#    json.dump(label_from_id_dict, outfile)   # Error: Object of type Label is not JSON serializable 


<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/labelbox/data/annotation_types/classification/classification.py:85: UserWarning: Dropdown classification is deprecated and will be removed in a future release
  warnings.warn("Dropdown classification is deprecated and will be "


# New Section

In [12]:
def handle_vertical_ht(x):
    if x=="BROKE":
        return 0
    elif type(x)==str and x.lower() in ["n/a","na","nan"]:
        return np.nan
    else:
        return float(x)

# Made Trial a str because it is not really being used as a numeric variable - better for plotting as it becomes a discrete variable instead of continuous (i.e. for color legend)
speculum_df_raw = pd.read_excel("data/01_raw/SpeculumTrialData_v2.xlsx", index_col=0, sheet_name="AllTrialsLongVals",
                                dtype={"Set Ct": np.int32, "Spec Ang": np.int32, "Spec Ht": np.int32, 
                                       #"Vertical Height": np.float64, 
                                       "Trial": str, "Filename": str, "Speculum Type": str},
                                converters={"Vertical Height": handle_vertical_ht},
                                )    
#key_cols = ["Speculum Type","Spec Ang","Spec Ht","Size","Material","Material Type","Method"]
#speculum_df_raw.drop_duplicates(subset=key_cols).reset_index().drop("index",axis=1).reset_index().rename({"index":"Set"},axis=1)
#set_info = speculum_df_raw[key_cols].drop_duplicates().reset_index().rename({"index":"Set"},axis=1)
#speculum_df_raw_with_set = speculum_df_raw.merge(set_info, how="outer",on=key_cols)


speculum_df_notfailed = speculum_df_raw[(speculum_df_raw["Used"] != "x")]   # Dropped the rows with failed trials

speculum_df_raw.to_csv("data/02_intermediate/speculum_df_raw"+".csv")
speculum_df_raw.to_pickle("data/02_intermediate/speculum_df_raw"+".pkl")
speculum_df_notfailed.to_csv("data/02_intermediate/speculum_df_notfailed"+".csv")
speculum_df_notfailed.to_pickle("data/02_intermediate/speculum_df_notfailed"+".pkl")

<IPython.core.display.Javascript object>

In [15]:
df_long=pd.merge(left=speculum_df_notfailed, right=labels_df, on="Filename")

glove_rows = df_long["Material Type"]=="Glove"
# The glove images got rotated 90 degrees. To fix this and clarify the directions of the opening, renaming the columns from x,y to wd and ht.
df_long.loc[ glove_rows,"wd"] = df_long.loc[ glove_rows].y
df_long.loc[ glove_rows,"ht"] = df_long.loc[ glove_rows].x
df_long.loc[ glove_rows,"wd_start"] = df_long.loc[ glove_rows].ystart
df_long.loc[ glove_rows,"ht_start"] = df_long.loc[ glove_rows].xstart

df_long.loc[~glove_rows,"wd"] = df_long.loc[~glove_rows].x
df_long.loc[~glove_rows,"ht"] = df_long.loc[~glove_rows].y
df_long.loc[~glove_rows,"wd_start"] = df_long.loc[~glove_rows].xstart
df_long.loc[~glove_rows,"ht_start"] = df_long.loc[~glove_rows].ystart
df_long = df_long.drop(columns=["x","y","xstart","ystart"])

df_long.head()

# Calculate relative value by dividing by the 0mmHg value
base_mmHg = 0 # mmHg
for ind in df_long["Trial Ct"].unique():
    df_long.loc[df_long["Trial Ct"]==ind,"wd_rel"]  = 1- df_long.loc[df_long["Trial Ct"]==ind].wd / df_long.loc[ (df_long["Trial Ct"]==ind) & (df_long["mmHg"]==base_mmHg) ].wd.item()
    df_long.loc[df_long["Trial Ct"]==ind,"ht_rel"]  = 1- df_long.loc[df_long["Trial Ct"]==ind].ht / df_long.loc[ (df_long["Trial Ct"]==ind) & (df_long["mmHg"]==base_mmHg) ].ht.item()
#df_long


<IPython.core.display.Javascript object>

In [18]:
df_multiindex = df_long.set_index(["Set Ct", "Set Trial Ct","mmHg"])
df_multiindex

<IPython.core.display.Javascript object>

Day Ct  Day Num  Trial Ct Speculum Type  Spec Ang  \
Set Ct Set Trial Ct mmHg                                                      
1      1            0          1        1         1         White         5   
                    40         1        2         1         White         5   
                    80         1        3         1         White         5   
                    120        1        4         1         White         5   
                    160        1        5         1         White         5   
...                          ...      ...       ...           ...       ...   
21     1            60         2       92        50         Green         5   
                    80         2       93        50         Green         5   
                    120        2       94        50         Green         5   
                    160        2       95        50         Green         5   
                    200        2       96        50         Green         5   

                          Spec Ht Size         Brand Material Material Type  \
Set Ct Set Trial Ct mmHg                                                      
1      1            0           0    M  STRONG-Black  Nitrile         Glove   
                    40          0    M  STRONG-Black  Nitrile         Glove   
                    80          0    M  STRONG-Black  Nitrile         Glove   
                    120         0    M  STRONG-Black  Nitrile         Glove   
                    160         0    M  STRONG-Black  Nitrile         Glove   
...                           ...  ...           ...      ...           ...   
21     1            60          0    M   STRONG-Blue  Nitrile         Glove   
                    80          0    M   STRONG-Blue  Nitrile         Glove   
                    120         0    M   STRONG-Blue  Nitrile         Glove   
                    160         0    M   STRONG-Blue  Nitrile         Glove   
                    200         0    M   STRONG-Blue  Nitrile         Glove   

                          ... Used  Notes  \
Set Ct Set Trial Ct mmHg  ...               
1      1            0     ...    y    NaN   
                    40    ...    y    NaN   
                    80    ...    y    NaN   
                    120   ...    y    NaN   
                    160   ...    y    NaN   
...                       ...  ...    ...   
21     1            60    ...    y    NaN   
                    80    ...    y    NaN   
                    120   ...    y    NaN   
                    160   ...    y    NaN   
                    200   ...    y    NaN   

                                                                        url  \
Set Ct Set Trial Ct mmHg                                                      
1      1            0     https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    40    https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    80    https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    120   https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    160   https://storage.labelbox.com/cl2ceiao35hbj0zah...   
...                                                                     ...   
21     1            60    https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    80    https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    120   https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    160   https://storage.labelbox.com/cl2ceiao35hbj0zah...   
                    200   https://storage.labelbox.com/cl2ceiao35hbj0zah...   

                                           Label ID     wd     ht wd_start  \
Set Ct Set Trial Ct mmHg                                                     
1      1            0     cl2cez4xr5ki20zagcvnuf7sk  257.0  261.0    585.0   
                    40    cl2cf020i5j8u0zdfhye4ehjg  190.0  287.0    614.0   
                    80    cl2cf0jjk5jak0zdf8

### Save processed dfs

In [19]:
df_long.to_csv(  "data/03_processed/combined_df_long"+".csv")
df_long.to_excel("data/03_processed/combined_df_long"+".xlsx")
df_long.to_pickle("data/03_processed/combined_df_long"+".pkl")

df_multiindex.to_excel("data/03_processed/combined_df_multiindex"+".xlsx")   # assuming a multiindex wouldn't save well to a csv file
df_multiindex.to_pickle("data/03_processed/combined_df_multiindex"+".pkl")  

<IPython.core.display.Javascript object>

## Get aggregate df across trials

In [33]:
# Group by all the parameters that will be the same across different trials of the same object
consistent_cols = ["Day Ct", "Set Ct", "Speculum Type", "Spec Ang", "Spec Ht", "Size", "Brand", "Material", "Material Type", "Method", "mmHg"]
aggregatable_cols = ["wd","ht","wd_rel","ht_rel", "Opening Height"]
grouped_trials = df_long[consistent_cols+aggregatable_cols].groupby(consistent_cols)
#display(grouped_trials.describe())

def sem(x, ddof=1):   # ddof=1 to get sample standard deviation, not the population standard deviation (np's default)
    sem = np.std(x, ddof=ddof)/np.sqrt(len(x))

def nonnan(x):
    return x[~np.isnan(x)]

df_agg_long = grouped_trials.agg([np.mean, scipy.stats.sem, np.std, np.min, np.median, np.max, np.count_nonzero], ddof=1).reset_index()

df_agg_long_flat = df_agg_long.copy()
df_agg_long_flat.columns = [".".join(col).strip(".") for col in df_agg_long.columns.values]
#df_agg_long_flat

df_agg_long.to_csv(   "data/04_aggregated/combined_df_agg_long"+".csv")
df_agg_long.to_excel( "data/04_aggregated/combined_df_agg_long"+".xlsx")
df_agg_long.to_pickle("data/04_aggregated/combined_df_agg_long"+".pkl")
df_agg_long_flat.to_csv("data/04_aggregated/combined_df_agg_long_flat"+".csv")
df_agg_long_flat.to_pickle("data/04_aggregated/combined_df_agg_long_flat"+".pkl")

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
df_agg_long_flat

<IPython.core.display.Javascript object>

,Day Ct,Set Ct,Speculum Type,Spec Ang,Spec Ht,Size,Brand,Material,Material Type,Method,...,ht_rel.median,ht_rel.amax,ht_rel.count_nonzero,Opening Height.mean,Opening Height.sem,Opening Height.std,Opening Height.amin,Opening Height.median,Opening Height.amax,Opening Height.count_nonzero
0,1,1,White,5,0,M,STRONG-Black,Nitrile,Glove,Middle,...,0.000000,0.000000,0,2.866667,0.166667,0.288675,2.7,2.7,3.2,3
1,1,1,White,5,0,M,STRONG-Black,Nitrile,Glove,Middle,...,0.090535,0.238739,3,2.866667,0.166667,0.288675,2.7,2.7,3.2,3
2,1,1,White,5,0,M,STRONG-Black,Nitrile,Glove,Middle,...,0.209877,0.279279,3,2.866667,0.166667,0.288675,2.7,2.7,3.2,3
3,1,1,White,5,0,M,STRONG-Black,Nitrile,Glove,Middle,...,0.042146,0.252252,3,2.866667,0.166667,0.288675,2.7,2.7,3.2,3
4,1,1,White,5,0,M,STRONG-Black,Nitrile,Glove,Middle,...,0.301802,0.403292,3,2.866667,0.166667,0.288675,2.7,2.7,3.2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,2,21,Green,5,0,M,STRONG-Blue,Nitrile,Glove,Middle,...,0.244444,0.329670,3,2.566667,0.066667,0.115470,2.5,2.5,2.7,3
93,2,21,Green,5,0,M,STRONG-Blue,Nitrile,Glove,Middle,...,0.333333,0.401099,3,2.566667,0.066667,0.115470,2.5,2.5,2.7,3
94,2,21,Green,5,0,M,STRONG-Blue,Nitrile,Glove,Middle,...,0.427778,0.472527,3,2.566667,0.066667,0.115470,2.5,2.5,2.7,3
95,2,21,Green,5,0,M,STRONG-Blue,Nitrile,Glove,Middle,...,0.477778,0.530220,3,2.566667,0.066667,0.115470,2.5,2.5,2.7,3


# Skip ahead from loaded code

In [22]:
speculum_df_raw = pd.read_pickle("data/02_intermediate/speculum_df_raw"+".pkl")
speculum_df_notfailed = pd.read_pickle("data/02_intermediate/speculum_df_notfailed"+".pkl")

labels_df = pd.read_csv("data/02_intermediate/labels_df.csv", index_col=0)
#with open("data/02_intermediate/label_from_id_dict"+".json", "r") as infile:
#    label_from_id_dict = json.load(infile)
    
df_long = pd.read_pickle(  "data/03_processed/combined_df_long.pkl")

df_agg_long = pd.read_pickle("data/04_aggregated/combined_df_agg_long.pkl")
df_agg_long_flat = pd.read_pickle("data/04_aggregated/combined_df_agg_long_flat.pkl")

df_multiindex = pd.read_pickle("data/03_processed/combined_df_multiindex"+".pkl")


<IPython.core.display.Javascript object>

# Analyze

In [23]:
df_agg_long

<IPython.core.display.Javascript object>

Speculum Type Spec Ang Spec Ht Size         Brand Material Material Type  \
                                                                              
0           Blue        5       0    M   STRONG-Blue  Nitrile         Glove   
1           Blue        5       0    M   STRONG-Blue  Nitrile         Glove   
2           Blue        5       0    M   STRONG-Blue  Nitrile         Glove   
3           Blue        5       0    M   STRONG-Blue  Nitrile         Glove   
4           Blue        5       0    M   STRONG-Blue  Nitrile         Glove   
..           ...      ...     ...  ...           ...      ...           ...   
80         White        5       0    S  STRONG-Black  Nitrile         Glove   
81         White        5       0    S  STRONG-Black  Nitrile         Glove   
82         White        5       0    S  STRONG-Black  Nitrile         Glove   
83         White        5       0    S  STRONG-Black  Nitrile         Glove   
84         White        5       0    S  STRONG-Black  Nitrile         Glove   

    Method mmHg          wd  ...    ht_rel                          \
                       mean  ...    median      amax count_nonzero   
0   Middle    0  237.666667  ...  0.000000  0.000000             0   
1   Middle   20  240.000000  ...  0.019231  0.071429             3   
2   Middle   40  219.333333  ...  0.079327  0.149770             3   
3   Middle   60  204.000000  ...  0.110577  0.216590             3   
4   Middle   80  179.333333  ...  0.161058  0.262673             3   
..     ...  ...         ...  ...       ...       ...           ...   
80  Middle   40  200.666667  ... -0.320930  0.029167             3   
81  Middle   80  140.000000  ... -0.186667  0.158333             3   
82  Middle  120   85.000000  ... -0.204651  0.262500             3   
83  Middle  160   61.333333  ... -0.086667  0.337500             3   
84  Middle  200   42.666667  ...  0.006667  0.420833             3   

   Opening Height                                                      
             mean       sem       std amin median  amax count_nonzero  
0        2.200000  0.152753  0.264575  2.0   2.10  2.50             3  
1        2.200000  0.152753  0.264575  2.0   2.10  2.50             3  
2        2.200000  0.152753  0.264575  2.0   2.10  2.50             3  
3        2.200000  0.152753  0.264575  2.0   2.10  2.50             3  
4        2.200000  0.152753  0.264575  2.0   2.10  2.50             3  
..            ...       ...       ...  ...    ...   ...           ...  
80       2.733333  0.016667  0.028868  2.7   2.75  2.75             3  
81       2.733333  0.016667  0.028868  2.7   2.75  2.75             3  
82       2.733333  0.016667  0.028868  2.7   2.75  2.75             3  
83       2.733333  0.016667  0.028868  2.7   2.75  2.75             3  
84       2.733333  0.016667  0.028868  2.7   2.75  2.75             3  

[85 rows x 44 columns]